<a href="https://colab.research.google.com/github/Priyali2000/Q-Commerce-Customer-Retention-Analysis/blob/main/01_Quick_Commerce_Customer_Retention_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>